
## Using a K-Mean Model to cluster London bicycle hires dataset


### Summary

This project uses a k-means model in BigQuery ML to identify clusters of data in the London Bicycle Hires public dataset.


### Key Concepts: 
- K-Means 
- Unsupervised models
- Geospatial analysis 
- Davies-Bouldin Index

## Objective 

- Create a binary K-means clustering model
- Make data-driven decisions based on BQML Visualization of the clusters


## Steps
1. Create the dataset to store the model 
1. Examine the training data 
1. Used the CREATE MODEL statement to create the the K-Means model 
1. Used the ML.PREDICT function to predict the station cluster. 
1. Use the model to make data-driven decisions to know which features are the most important to determine the income bracket.


#### Execute notebook in Colab
<a href="https://colab.research.google.com/github/paulycloud/ml_portfolio/blob/main/02_BigQuery_ML/04_london_bike_hires_k_means/index.ipynb">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>


### Dataset

The dataset is the [`London Bicycle Hires public`](https://cloud.google.com/marketplace/details/greater-london-authority/london-bicycles?filter=solution-type:dataset&id=95374cac-2834-4fa2-a71f-fc033ccb5ce4&_ga=2.18644735.777390235.1667764380-77713112.1665872214&_gac=1.186188379.1667497020.CjwKCAjwzY2bBhB6EiwAPpUpZhvKzZjSLlQgJkUZn1ZcM1bGDnIbpyv8CT3SHbte7N3PEvRI58Vg0BoCLbkQAvD_BwE). The London Bicycle Hires data contains the number of hires of London's Santander Cycle Hire Scheme from 2011 to present. The dataset  has about 24,369,201 rows of data. 

The data includes: 
*   start and stop timestamps
*   station names 
*   ride duration


The queries in this tutorial use Geography Functions available in [**geospatial analytics**](https://cloud.google.com/bigquery/docs/gis-intro)


### Install additional packages

Install the following packages required to execute this notebook. 

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q google-cloud-bigquery db-dtypes

     |████████████████████████████████| 2.3 MB 2.6 MB/s 
     |████████████████████████████████| 211 kB 36.0 MB/s 
     |████████████████████████████████| 47 kB 1.3 MB/s 
     |████████████████████████████████| 233 kB 10.4 MB/s 
     |████████████████████████████████| 206 kB 22.4 MB/s 
     |████████████████████████████████| 408 kB 47.9 MB/s 
     |████████████████████████████████| 106 kB 4.2 MB/s 
     |████████████████████████████████| 115 kB 9.0 MB/s 
     |████████████████████████████████| 77 kB 1.8 MB/s 
     |████████████████████████████████| 1.0 MB 33.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.13.3 requires google-cloud-bigquery[

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Project Variables 

In [18]:
# Project variables 
#
# These are the project variable used in this ML Model: 
#

PROJECT_ID = "" # @param {type:"string"}
bqml_type = "kmeans" # @param {type:"string"}
BQML_MODEL_NAME = "bqml_kmeans_london_stations_cluster_model"
job_display_name = BQML_MODEL_NAME + "_job"
ENDPOINT_NAME = BQML_MODEL_NAME + "_endpoint"

# datasets
BQ_DATASET_NAME = "03_bqml_k_means_clustering_bike"
sql_create_dataset = f"""CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}"""
BQ_PUBLIC_DATASET_HIRES = "bigquery-public-data.london_bicycles.cycle_hire"
BQ_PUBLIC_DATASET_STATIONS = "bigquery-public-data.london_bicycles.cycle_stations"

# bucket details
BUCKET_NAME = "bqml_tutorials"
BUCKET_URI = f"gs://{BUCKET_NAME}/{bqml_type}/"
OUTPUTBUCKET = f"gs://bqml_datasets_predictions/{bqml_type}/"

# Region 
REGION = "us-central1" # @param {type: "string"} 

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You might not be able to use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/general/locations" target="_blank">Vertex AI regions</a>.

In [2]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [3]:
SERVICE_ACCOUNT = ""  # @param {type:"string"}
print(SERVICE_ACCOUNT)

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Import libraries

In [10]:
from typing import Union

import google.cloud.aiplatform as vertex_ai
import pandas as pd
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [11]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

Create the BigQuery client.

In [12]:
bq_client = bigquery.Client(project=PROJECT_ID)

Use a helper function for sending queries to BigQuery.

In [13]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

## BigQuery ML Model Training & Validation

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification, regression, forecasting, and matrix factorization, in BigQuery using SQL syntax directly. BigQuery ML uses the scalable infrastructure of BigQuery ML so you don't need to set up additional infrastructure for training or batch serving.

### Create the dataset
The first step is to create a BigQuery dataset to store your model. To create your dataset:

In [ ]:
sql_create_dataset = f"""
CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}
OPTIONS (
    location = 'europe-west4'
  )
"""

print(sql_create_dataset)

run_bq_query(sql_create_dataset)

### Examine the training data
The “`london_bicycles`” table contains the data needed. 

Because **k-means** is an **unsupervised** learning technique, model training does not require labels nor does it require to split the data into training data and evaluation data. The following query compiled the training data.

In [ ]:
sql_inspect = f"""
WITH
 hs AS (
 SELECT
   h.start_station_name AS station_name,
   IF
   (EXTRACT(DAYOFWEEK
     FROM
       h.start_date) = 1
     OR EXTRACT(DAYOFWEEK
     FROM
       h.start_date) = 7,
     "weekend",
     "weekday") AS isweekday,
   h.duration,
   ST_DISTANCE(ST_GEOGPOINT(s.longitude,
       s.latitude),
     ST_GEOGPOINT(-0.1,
       51.5))/1000 AS distance_from_city_center
 FROM
   {BQ_PUBLIC_DATASET_HIRES} AS h
 JOIN
   {BQ_PUBLIC_DATASET_STATIONS} AS s
 ON
   h.start_station_id = s.id
 WHERE
   h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
   AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
 stationstats AS (
 SELECT
   station_name,
   AVG(duration) AS duration,
   COUNT(duration) AS num_trips,
   MAX(distance_from_city_center) AS distance_from_city_center
 FROM
   hs
 GROUP BY
   station_name )
SELECT
 *
FROM
 stationstats
ORDER BY
 distance_from_city_center ASC

"""

run_bq_query(sql_inspect)

### Used the CREATE MODEL statement to create the the K-Means model

When the model is created, the clustering field is station_name, and cluster the data based on station attribute, for example the distance of the station from the city center.

In the `OPTIONS` parameter:
* with `model_registry="vertex_ai"`, the BigQuery ML model will automatically be <a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-registry-bqml" target="_blank">registered to Vertex AI Model Registry</a>, which enables you to view all of your registered models and its versions on Google Cloud in one place.

* `vertex_ai_model_version_aliases allows you to set aliases to help you keep track of your model version (<a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-alias" target="_blank">documentation</a>).

Clustering of bike stations was based on the following attributes:

- Duration of rentals
- Number of trips per day
- Distance from city center



In [ ]:
# this cell may take ~1 min to run

sql_train_model_bqml = f"""

CREATE
OR REPLACE MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME} OPTIONS(
    model_type = 'kmeans',
    num_clusters = 4,
    model_registry = "vertex_ai",
    vertex_ai_model_version_aliases = ['kmeans', 'experimental']
) AS WITH hs AS (
    SELECT
        h.start_station_name AS station_name,
        IF (
            EXTRACT(
                DAYOFWEEK
                FROM
                    h.start_date
            ) = 1
            OR EXTRACT(
                DAYOFWEEK
                FROM
                    h.start_date
            ) = 7,
            "weekend",
            "weekday"
        ) AS isweekday,
        h.duration,
        ST_DISTANCE(
            ST_GEOGPOINT(s.longitude, s.latitude),
            ST_GEOGPOINT(-0.1, 51.5)
        ) / 1000 AS distance_from_city_center
    FROM
        { BQ_PUBLIC_DATASET_HIRES } AS h
        JOIN { BQ_PUBLIC_DATASET_STATIONS } AS s ON h.start_station_id = s.id
    WHERE
        h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
        AND CAST('2016-01-01 00:00:00' AS TIMESTAMP)
),
stationstats AS (
    SELECT
        station_name,
        isweekday,
        AVG(duration) AS duration,
        COUNT(duration) AS num_trips,
        MAX(distance_from_city_center) AS distance_from_city_center
    FROM
        hs
    GROUP BY
        station_name,
        isweekday
)
SELECT
    *
EXCEPT
(station_name, isweekday)
FROM
    stationstats
"""

print(sql_train_model_bqml)

run_bq_query(sql_train_model_bqml)

## Results 

The results matrix show the *Davies-Bouldin Index* and the *Mean Squared Distance.*

**Davies-Bouldin index** is a validation metric that is often used in order to evaluate the optimal number of clusters to use. It is defined as a ratio between the cluster scatter and the cluster’s separation and a lower value will mean that the clustering is better.

The **mean squared distance** makes reference to the intra cluster variance, which we want to minimize as a lower WCSS (within-cluster sums of squares) will maximize the distance between clusters.

The Numerical Features tab displays visualizations of the clusters identified by the k-means model. Under Numerical features, bar graphs display up to 10 of the most important numerical feature values for each centroid.



### Step 4: Used the ML.PREDICT function to predict the station cluster.

The ML.PREDICT function was used to predict the cluster for a given set of stations. You predict clusters for all station names that contain the string Kennington.

In [ ]:
sql_ml_predict = f"""
 WITH
 hs AS (
 SELECT
   h.start_station_name AS station_name,
   IF
   (EXTRACT(DAYOFWEEK
     FROM
       h.start_date) = 1
     OR EXTRACT(DAYOFWEEK
     FROM
       h.start_date) = 7,
     "weekend",
     "weekday") AS isweekday,
   h.duration,
   ST_DISTANCE(ST_GEOGPOINT(s.longitude,
       s.latitude),
     ST_GEOGPOINT(-0.1,
       51.5))/1000 AS distance_from_city_center
 FROM
   {BQ_PUBLIC_DATASET_HIRES} AS h
 JOIN
   {BQ_PUBLIC_DATASET_STATIONS} AS s
 ON
   h.start_station_id = s.id
 WHERE
   h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
   AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
 stationstats AS (
 SELECT
   station_name,
   AVG(duration) AS duration,
   COUNT(duration) AS num_trips,
   MAX(distance_from_city_center) AS distance_from_city_center
 FROM
   hs
 GROUP BY
   station_name )
SELECT
 * EXCEPT(nearest_centroids_distance)
FROM
 ML.PREDICT( MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
   (
   SELECT
     *
   FROM
     stationstats
   WHERE
     REGEXP_CONTAINS(station_name, 'Kennington')))
"""

print(sql_ml_predict)

run_bq_query(sql_ml_predict)

### Use the ML.EXPLAIN_PREDICT function to know which features are the most important to determine the weight.


To understand why the model is generating these prediction results, you can use the ML.EXPLAIN_PREDICT function.


<a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a> has built-in <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview" target="_blank">Explainable AI</a>. This allows you to see the top contributing features to each prediction and interpret how it was computed.

In [ ]:
sql_explain_predict = f"""

SELECT * FROM
ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
 (
  WITH
 hs AS (
 SELECT
   h.start_station_name AS station_name,
   IF
   (EXTRACT(DAYOFWEEK
     FROM
       h.start_date) = 1
     OR EXTRACT(DAYOFWEEK
     FROM
       h.start_date) = 7,
     "weekend",
     "weekday") AS isweekday,
   h.duration,
   ST_DISTANCE(ST_GEOGPOINT(s.longitude,
       s.latitude),
     ST_GEOGPOINT(-0.1,
       51.5))/1000 AS distance_from_city_center
 FROM
   {BQ_PUBLIC_DATASET_HIRES} AS h
 JOIN
   {BQ_PUBLIC_DATASET_STATIONS} AS s
 ON
   h.start_station_id = s.id
 WHERE
   h.start_date BETWEEN CAST('2015-01-01 00:00:00' AS TIMESTAMP)
   AND CAST('2016-01-01 00:00:00' AS TIMESTAMP) ),
 stationstats AS (
 SELECT
   station_name,
   AVG(duration) AS duration,
   COUNT(duration) AS num_trips,
   MAX(distance_from_city_center) AS distance_from_city_center
 FROM
   hs
 GROUP BY
   station_name )
SELECT
 * EXCEPT(nearest_centroids_distance)
 )
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)

### Inspect the model on Vertex AI Model Registry

When the model was trained in BigQuery ML, the line `model_registry="vertex_ai"` registered the model to Vertex AI Model Registry automatically upon completion.

You can view the model on the <a href="https://console.cloud.google.com/vertex-ai/models" target="_blank">Vertex AI Model Registry page</a>, or use the code below to check that it was successfully registered:

In [ ]:
model = vertex_ai.Model(model_name=BQML_MODEL_NAME)

print(model.gca_resource)

### Deploy the model to an endpoint

While BigQuery ML supports batch prediction with <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">ML.PREDICT</a> and <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a>, BigQuery ML is not suitable for real-time predictions where you need low latency predictions with potentially high frequency of requests.

In other words, deploying the BigQuery ML model to an endpoint enables you to do online predictions.

#### Create a Vertex AI endpoint

To deploy your model to an endpoint, you will first need to create an endpoint before you deploy the model to it.

In [ ]:

endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

#### List endpoints

List the endpoints to make sure it has successfully been created. (You can also view your endpoints on the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>).

In [ ]:
endpoint.list()

#### Deploy model to Vertex endpoint

With the new endpoint, you can now deploy your model.

In [ ]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

You can also check on the status of your model by visiting the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>.

### Make online predictions to the endpoint

Using a sample of the training data, you can test the endpoint to make online predictions.

In [ ]:
df_sample_requests_list = [
    {
        "age": 45,
        "workclass": "Private",
        "marital_status": "Single",
        "education_num": 6,
        "occupation": "Exec-managerial",
        "hours_per_week": 40,
    },
    {
        "age": 30,
        "workclass": "Private",
        "marital_status": "Married",
        "education_num": 2,
        "occupation": "Machine-op-inspct",
        "hours_per_week": 50,
    }
]

In [ ]:
prediction = endpoint.predict(df_sample_requests_list)
print(prediction)

You can then extract the predictions from the prediction response

In [ ]:
prediction.predictions

## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model from endpoint and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

# Delete BigQuery dataset, including the BigQuery ML model
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME